This notebook is used to create a World builder file using an input JSON file. Specifically, we use this notebook for generating world builder files describing plate boundaries.
The notebook is originally written by Richard Styron.

In [1]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("error")
import matplotlib.pyplot as plt

In [2]:
#--------------------------------------------------------------------
# Input the data file in JSON here defining the plate boundaries. 
# There are four options to choose from in the current directory
#   1) gem_active_faults.geojson : Contains all faults from the Global
#      Earthquake model
#   2) bird_gem_faults.json : Contains only faults from the Peter Bird's
#      2003 catalog using the Global Earthquake model
#   3) PB2002_steps.json : Contains faults defined by Peter Bird's 
#      plate boundary model. This file has closed polygons and are 
#      vertically dipping.
#--------------------------------------------------------------------

plate_boundary_model = "./bird_gem_faults.json"

In [3]:
with open(plate_boundary_model) as f:
    gg = json.load(f)

In [4]:
faults = gg['features']
faults = [f for f in faults]

In [14]:
EARTH_RADIUS = 6371.0

def val_from_tup_str(tup_str):
    """
    We reformat the input json file to remove parenthesis and use comma to
    separate fields. This is done because the input json filed based on the
    GEM model has some keys with values containting paranthesis and multiple 
    commas (e.g., "strike_slip_rate" : "(20,,)"). We use the function to 
    extract only the relevant value (i.e, 20), which is later used by other
    functions for computation of fault dip and fault length.
    """
    tup_str = tup_str.replace('(', '').replace(')', '')
    vals = tup_str.split(',')
    val = vals[0]
    return val


def azimuth(lon1, lat1, lon2, lat2):
    """
    Calculate the azimuth between two points or two collections of points.
    Parameters are the same as for :func:`geodetic_distance`.
    Implements an "alternative formula" from
    http://williams.best.vwh.net/avform.htm#Crs
    :returns:
        Azimuth as an angle between direction to north from first point and
        direction to the second point measured clockwise in decimal degrees.
    """
    #lon1, lat1, lon2, lat2 = _prepare_coords(lon1, lat1, lon2, lat2)
    cos_lat2 = np.cos(lat2)
    true_course = np.degrees(np.arctan2(
        np.sin(lon1 - lon2) * cos_lat2,
        np.cos(lat1) * np.sin(lat2)
        - np.sin(lat1) * cos_lat2 * np.cos(lon1 - lon2)
    ))
    return (360 - true_course) % 360


def geodetic_distance(lons1, lats1, lons2, lats2, diameter=2*EARTH_RADIUS):
    """
    Calculate the geodetic distance between two points or two collections
    of points.
    Parameters are coordinates in decimal degrees. They could be scalar
    float numbers or np arrays, in which case they should "broadcast
    together".
    Implements http://williams.best.vwh.net/avform.htm#Dist
    :returns:
        Distance in km, floating point scalar or np array of such.
    """
    distance = np.arcsin(np.sqrt(
        np.sin((lats1 - lats2) / 2.0) ** 2.0
        + np.cos(lats1) * np.cos(lats2)
        * np.sin((lons1 - lons2) / 2.0) ** 2.0
    ))
    return diameter * distance


def average_azimuth(coords):
    """
    Calculate and return weighted average azimuth of all line's segments
    in decimal degrees.
    Uses formula from
    http://en.wikipedia.org/wiki/Mean_of_circular_quantities
    >>> from openquake.hazardlib.geo.point import Point as P
    >>> '%.1f' % Line([P(0, 0), P(1e-5, 1e-5)]).average_azimuth()
    '45.0'
    >>> '%.1f' % Line([P(0, 0), P(0, 1e-5), P(1e-5, 1e-5)]).average_azimuth()
    '45.0'
    >>> line = Line([P(0, 0), P(-2e-5, 0), P(-2e-5, 1.154e-5)])
    >>> '%.1f' % line.average_azimuth()
    '300.0'
    """
    if len(coords) == 2:
        return azimuth(coords[0][0], coords[0][1], coords[1][0], coords[1][1])
    
    lons = np.array([c[0] for c in coords])
    lats = np.array([c[1] for c in coords])
    azimuths = azimuth(lons[:-1], lats[:-1], lons[1:], lats[1:])
    distances = geodetic_distance(lons[:-1], lats[:-1],
                                  lons[1:], lats[1:])
    azimuths = np.radians(azimuths)
    # convert polar coordinates to Cartesian ones and calculate
    # the average coordinate of each component
    avg_x = np.mean(distances * np.sin(azimuths))
    avg_y = np.mean(distances * np.cos(azimuths))
    # find the mean azimuth from that mean vector
    az = np.degrees(np.arctan2(avg_x, avg_y))
    if az < 0:
        az += 360
    return az


direction_map = {'N': 0.,
                 'NNE': 22.5,
                 'NE': 45.,
                 'ENE': 67.5,
                 'E': 90.,
                 'ESE': 112.5,
                 'S': 180.,
                 'W': 270.,
                 'NW': 315.,
                 'SE': 135.,
                 'SW': 225.,
                 'U': 0.}


def check_right_hand_rule(trace, dip_dir):
    strike = average_azimuth(trace)

    trace_dip_trend = strike + 90.

    fault_dip_trend = direction_map[dip_dir]

    trend_angle_diff = angle_difference(fault_dip_trend, trace_dip_trend)

    if abs(90 - trend_angle_diff) < 15:
        warnings.warn('Given dip direction <15 degrees of strike')

    if trend_angle_diff > reverse_angle_threshold:
        new_fault_trace = fault_trace[::-1]
        return new_fault_trace
    else:
        return fault_trace
    
    
dip_map = {'Normal': 60.,
           'Normal-Dextral': 65.,
           'Normal-Sinistral': 65.,
           'Reverse': 40.,
           'Reverse-Dextral': 65.,
           'Reverse-Sinistral': 65.,
           'Reverse-Strike-Slip': 65.,
           'Sinistral': 90.,
           'Sinistral-Normal': 65.,
           'Sinistral-Reverse': 65.,
           'Dextral': 90.,
           'Dextral-Reverse': 65.,
           'Dextral-Normal': 65.,
           'Strike-Slip': 90.,
           'Thrust': 40.,
           'Blind_Thrust': 40.,
           'Blind Thrust': 40.,
           'Subduction_Thrust': 20.,
           'Spreading_Ridge': 60.,
           'Sinistral_Transform': 90.,
           'Dextral_Transform': 90.}
    
    


In [20]:
def az_to_angle(az):
    return np.radians(az - 90.)


def check_vertical_fault(fault):
    if "dip_dir" not in fault["properties"].keys():
        if fault["properties"]["slip_type"] in ["Sinistral", "Dextral", "Sinistral_Transform",
                                                "Dextral_Transform", "Spreading_Ridge"]:
            return True
        else:
            return False
    else:
        return False
        

def get_dip_point(fault):
    
    strike = average_azimuth(fault['geometry']['coordinates'])
    trace_dip_trend = strike + 90.
    
    # check right hand rule
    try:
        fault_dip_trend = direction_map[fault['properties']['dip_dir']]
    
        trend_angle_diff = angle_difference(fault_dip_trend, trace_dip_trend)
        if trend_angle_diff > 90:
            trace_dip_trend = strike + 270
    except:
        pass
        
    dip_trend_angle = az_to_angle(trace_dip_trend)
    
    x, y = np.sin(dip_trend_angle), np.cos(dip_trend_angle)
    
    dip_point = [x,y]
    
    return dip_point    


def get_dip(fault):
    try:
        dip = float(val_from_tup_str(fault['properties']['average_dip']))
        if not 0 < dip <= 90.:
            dip = dip_map[fault["properties"]["slip_type"]]
    except:
        dip = 90 # change here by AS for using default boundaries to be vertical
    return dip
        
        
        
def get_length(fault, default_lsd = 20.):
    dip = get_dip(fault)
    if dip is None:
        return
    
    try:
        usd = float(val_from_tup_str(fault['properties']['upper_seis_depth']))
        lsd = float(val_from_tup_str(fault['properties']['lower_seis_depth']))
    except:
        usd = 0.
        lsd = default_lsd
    
    # In the input gem_active_faults.geojson file, some faults have zero dip, 
    # which will lead to undefined length. There are also few faults with 
    # 'upper_seis_depth' 0 or nan. For these faults, world builder crashes.
    # We set the length of the faults to a default 20 km, this the average
    # length of all the faults in the database.
    default_length = 20.
    if (dip != 0):    
        length = (lsd - usd) / np.sin(np.radians(dip))
    
    if (length != length or length == 0):
        length = default_length 
   
    return length

In [26]:
def make_fault_json(feat, fault_number, thickness=100e3):
    
    # There are some faults in gem_active_faults.geojson file with fault coordinates
    # that are consecutively same. This gives an error in world builder, therefore
    # we use unique coordinates only. 
    input_coordinates       = feat["geometry"]["coordinates"]
    input_coordinates_array = np.array(input_coordinates)
    unique_coordinates      = np.unique(input_coordinates_array, axis=0)
    
    # There are also few faults with coordinates as (longitude, latitude, depth). 
    # Since we only need to give coordinates of the fault trace in 
    # worldbuilder, we take the first two values in the coordinates.
    if len(input_coordinates[0]) == 3:
        coordinates_output = input_coordinates_array[:, 0:2]
    else:
        coordinates_output = input_coordinates_array
    
    props = feat["properties"]

    # We use 300 km for length (equivalent to depth in the world builder terminology)
    # for defining plate boundaries such as faults in Bird's catalog. This is capped 
    # by the lithospheric depths in our material model. 
    # We use the seismicity based lengths using get_length() for diffused zones in 
    # the Global Earthquake Model.
    out = {
        "model": "fault",
        "name" : "fault" + "_" + str(fault_number),
        "coordinates": coordinates_output.tolist(),
        "dip point": get_dip_point(feat),
        "min depth": -1e3,
        "max depth": 300e3,
        "segments": [
            {
                "length": get_length(feat)*1e3,
#                 "length": 300e3,
                "thickness": [100e3],
                "angle": [get_dip(feat), get_dip(feat)]   
            }
        ],
        # temperature, composition, etc. would go here"
        # max depth chosen as the max continental plate thickness
        # hard coded the composition here
        "composition models": [{"model":"smooth", "compositions":4, "side distance fault center":50e3}] 
    }

    return out

In [39]:
# This code cell takes all the faults from a json file and stores them as features, 
# the format that the worldbuilder needs. 
# The next code cell adds the header of the worldbuilder file, and the next cell writes 
# everything into the worldbuilder file.
fault_count = 1
fault_json =[]

for fault in faults:
    fault_json.append(make_fault_json(fault, fault_count))
    fault_count = fault_count + 1

In [12]:

all_faults_json = {
    "version": "0.5",
    "interpolation": "continuous monotone spline",
    "coordinate system":{"model":"spherical", "depth method":"starting point"},
    "features": fault_json
}

In [13]:
with open("./input_data/geodynamic_world_builder_bird_boundaries_closed_polygons.json", "w") as f:
    json.dump(all_faults_json, f)

In [5]:
# This cell outputs the coordinates in a text file, so that we can use
# them later for plotting in GMT.
# We use the same file name as the input json but with a .txt extension.
filename_in_gmt = plate_boundary_model.split('.')[1][1:] + '.txt'

with open (filename_in_gmt, "w") as f:
    for i in range(len(faults)):
        coordinates = faults[i]["geometry"]["coordinates"]
        f.writelines("%s\n" % coordinate_line for coordinate_line in coordinates)
    f.close()

In [28]:
# This cell outputs the length from the get_length() only used for 
# debugging errors in Worldbuilder while using 'gem_active_faults.geojson', 
# not used otherwise.
length_fault  = []
length_fault= np.ones((len(faults,)))

for fault in faults:
    length_fault[fault_count-1] = get_length(fault) 

20.0